# Finetuning our model

In our previous notebook we [built a simple fashion search engine using Docarray](https://colab.research.google.com/github/alexcg1/neural-search-notebooks/blob/main/fashion-search/1_build_basic_search/basic_search.ipynb).

Now we'll finetune our model to deliver better results!

## Setup

In [ ]:
!pip install torchvision~=0.11

In [ ]:
!pip install git+https://github.com/jina-ai/finetuner # Change to stable release later

In [ ]:
from docarray import Document, DocumentArray

## Load images

This is just the same process we followed in the last notebook

In [ ]:
DATA_DIR = "./data"
DATA_PATH = f"{DATA_DIR}/*.jpg"
MAX_DOCS = 1000

# Toy data - If data dir doesn't exist, we'll get data of ~800 fashion images from here
TOY_DATA_URL = "https://github.com/alexcg1/neural-search-notebooks/blob/main/fashion-search/data.zip?raw=true"

In [ ]:
# Download images if they don't exist
import os

if not os.path.isdir(DATA_DIR) and not os.path.islink(DATA_DIR):
    print(f"Can't find {DATA_DIR}. Downloading toy dataset")
    !wget "$TOY_DATA_URL" -O data.zip
    !unzip -q data.zip # Don't print out every darn filename
    !rm -f data.zip
else:
    print(f"Nothing to download. Using {DATA_DIR} for data")

In [ ]:
docs = DocumentArray.from_files(DATA_PATH, size=MAX_DOCS)
print(f"{len(docs)} Documents in DocumentArray")

In [ ]:
def preproc(doc):
    return (
        doc.load_uri_to_image_tensor(80, 60)
        .set_image_tensor_normalization()
        .set_image_tensor_channel_axis(-1, 0)
    )


docs.apply(preproc)

## Load model

Again, we're playing the same old song, loading a model just like we did last time.

In [ ]:
import torchvision

model = torchvision.models.resnet50(pretrained=True)
# model = torchvision.models.resnet18(pretrained=True)

In [ ]:
# Let's look at the layers
import finetuner as ft
ft.display(model, (3, 80, 60))

## Finetune model

Here's where the new stuff kicks in!

We'll:

- Set some basic parameters
- Install a module to see progress
- Finetune our model, focusing on the embedding layer *just* before the classification layer

In [ ]:
import torch

# Basic setup
EPOCHS = 6         # higher = more time, better finetuning
BATCH_SIZE = 64    # higher = use more memory
# LAYER_NAME = "adaptiveavgpool2d_67" # for resnet18
LAYER_NAME = "adaptiveavgpool2d_173" # for resnet50

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
# See progress bar in notebook
!pip install -q ipywidgets # -q = quiet
import ipywidgets

While tuning, keep an eye on the loss rate. It starts at about 0.36 then falls to about 0.08 after 6 epochs.

In [ ]:
tuned_model = ft.fit(
    model=model,
    train_data=docs,
    loss='TripletLoss',
    epochs=EPOCHS,
    device=DEVICE,
    batch_size=BATCH_SIZE,
    to_embedding_model=True,
    input_size=(3, 80, 60),
    layer_name=LAYER_NAME, # layer before fc as feature extractor
    freeze=False,
)

## Save model

In [ ]:
import torch

torch.save(tuned_model, "tuned-model")

In [ ]:
# If running in Colab, download to local filesystem
try:
    from google.colab import files
    files.download("tuned-model")
except:
    pass

## Next steps

Next we'll:

- Load the tuned model into our original script
- Compare results with the base model